#  **Extração das respostas**


**Authors:** Renato Freitas Bessa, Adonias Caetano de Oliveira, Ariel Soares Teles

**Institution:** UFDPar - PPGBiotec

**Objective:** extrai a alternativa indicada por cada resposta por meio de expressões regulares (2º tentativa de extração)

In [18]:
import pandas as pd
import re
import random
import numpy as np

## **Loading data**

In [19]:
#sheet_names = ["2017.01", "2017.02", "2020.01", "2020.02", "2021.01", "2022.01", "2022.02", "2023.01", "2023.02"]
sheet_names = ["2024.01"]

In [ ]:
url = '<public link to access the answer keys spreadsheet>'
file_id = url.split('/')[-2]
read_url='https://drive.google.com/uc?id=' + file_id

# read the data
gabaritos = {}

for name in sheet_names:
  gabaritos[name] = pd.read_excel(read_url,  sheet_name=name)

In [21]:

def extrair_opcao(resposta):

    patterns = [
        r"A opção correta para essa situação é a \*\*([a-dA-D])",
        r"o item correto é:\s*([a-dA-D])",
        r"para este caso é:\s*([a-dA-D])",
        r"a opção correta é \*\*([a-dA-D])",
        r"seria a opção ([a-dA-D])",
        r"é a opção \*\*([a-dA-D])",
        r"é a opção:\s*([a-dA-D])",
        r"é a alternativa \*\*([a-dA-D])",
        r"a alternativa correta é:\s*([a-dA-D])",
        r"a opção correta é:\s*([a-dA-D])",
        r"A opção mais adequada parece ser a \*\*([a-dA-D])",
        r"resposta correta é ([a-dA-D])",
        r"resposta é ([a-dA-D])",
        r"Resposta: ([a-dA-D])",
        r"Resposta correta: ([a-dA-D])",
        r"alternativa correta é ([a-dA-D])",
        r"correta é ([a-dA-D])",
        r"correta é a letra ([a-dA-D])",
        r"correcta é ([a-dA-D])",
        r"correcta é a letra ([a-dA-D])",
        r"A resposta ([a-dA-D])",
        r"resposta correta seria ([a-dA-D])",
        r"é a \"\"([a-dA-D])\"\"",
        r"seria \"\"([a-dA-D])\"\"",
        r"a resposta mais comum seria '([a-dA-D])'",
        r"resposta correta é: ([a-dA-D])",
        r"A alternativa mais alinhada com .*? é a ([a-dA-D])",
        r"tornando a opção \(([a-dA-D])\) a correta",
        r'a opção \"\"([a-dA-D])\"\" parece ser a mais adequada',
        r"a opção ([a-dA-D]) é a mais adequada",
        r"Portanto, a opção ([a-dA-D]) é a mais adequada",
        r"a opção \*\*([a-dA-D])\.", # Removi o .*?\*\*
        r"Posto isto, a opção mais adequada é a ([a-dA-D])",
        r"a menos incorreta dadas as opções apresentadas é a '([a-dA-D])'",
        r'Portanto, a alternativa \"([a-dA-D])\" é a mais apropriada',
        r"Portanto, a opção \*\*([a-dA-D])\.", # Removi o .*?\*\*
        r"alternativa ([a-dA-D]) é a mais apropriada",
        r"Portanto, a conduta delineada na alternativa ([a-dA-D]) é a mais apropriada",
        r'Portanto, a opção \"([a-dA-D])\" é a mais adequada',
        r"\(alternativa ([a-dA-D])\)",
        r"O diagnóstico mais provável é:\s*([a-dA-D])",
        r"A conduta médica mais adequada, dada a situação apresentada, é a opção ([a-dA-D])",
        r"Diante dos dados, a opção \*\*([a-dA-D])\.", # Removi o .*?\*\*
        r"a conduta indicada é a letra ([a-dA-D])",
        r"item ([a-dA-D]) é o correto",
        r"Portanto, a alternativa correta é \*\*([a-dA-D])",
        r"O item correto é ([a-dA-D])\.",
        r"a opção que mais se alinha com os sintomas e resultados é a letra \*\*([a-dA-D])",
        r"a indicação mais correta seria realizar uma \*\*[a-zA-Z]+\*\* \(opção ([a-dA-D])\)",
        r"justificando a escolha da opção ([a-dA-D])",
        r"na questão é mais indicativo de [a-z\sA-Z]+ \(item ([a-dA-D])\)",
        r"Portanto, a melhor conduta neste caso é a opção ([a-dA-D])",
        r"A hipótese diagnóstica mais provável para o caso descrito é a ([a-dA-D])",
        r"A principal hipótese diagnóstica, de acordo com o quadro clínico descrito, é:\s*([a-dA-D])\.",
        r"Assim, a opção que mais aproxima do manejo em casos de dúvida diagnóstica de placenta prévia seria:\s*\*\*([a-dA-D])",
        r"A opção mais alinhada com práticas comuns e pragmáticas, considerando o cenário descrito, é a ([a-dA-D])",
        r"Considerando todos esses aspectos, a opção que mais se alinha com o quadro clínico apresentado é a opção ([a-dA-D])",
        r"A conduta terapêutica mais adequada para fraturas do colo do fêmur estágio II costuma ser:\s*([a-dA-D])",
        r"Portanto, a alternativa \*\*([a-dA-D])\.", # Removi o .*?\*\*
        r"A principal hipótese diagnóstica para este caso seria o \*\*[a-zA-Z]+ \(opção ([a-dA-D])\)\*\*",
        r"A opção correta, considerando as informações disponíveis na questão, é a \*\*([a-dA-D])",
        r"Diante disso, entre as opções oferecidas, a medida inicial mais apropriada seria:\s*([a-dA-D])\.",
        r"Portanto, a opção ([a-dA-D]) é a correta\.",
        r"Portanto, o item correto é a opção ([a-dA-D])",
        r"Dado isso, a \*\*alternativa ([a-dA-D])\*\* parece ser a mais correta",
        r"Dado o contexto dos achados laboratoriais e clínicos, a opção \*\*([a-dA-D])\.", # Removi o .*?\*\*
        r"Essas características apontam para ações de natureza \*\*terciária\*\* \(opção ([a-dA-D])\)",
        r"Com base nas informações apresentadas, a opção \*\*([a-dA-D])\*\*",
        r"O diagnóstico mais provável para a condição da mulher descrita na questão é:\s*([a-dA-D])\.",
        r"Portanto, a opção mais indicada é a \*\*([a-dA-D])\)",
        r"Com base nos dados fornecidos, a explicação mais provável para .*? é:\s*([a-dA-D])",
        r"No caso clínico apresentado, a conduta indicada é:\s*([a-dA-D])\.",
        r"Portanto, a alternativa \*\*([a-dA-D])\*\* está correta",
        r"Diante das opções apresentadas, a alternativa \*\*([a-dA-D])\*\* é a mais adequada",
        r"Portanto, a resposta correta é:\s*\*\*([a-dA-D])",
        r"Portanto, a conduta mais apropriada é a descrita na \*\*opção ([a-dA-D])\*\*",
        r"Portanto, a alternativa \*\*([a-dA-D])\*\* é a mais apropriada",
        r"O quadro clínico descrito é mais indicativo de leishmaniose visceral \(item ([a-dA-D])\)",
        r"Considerando essas análises, a opção \*\*([a-dA-D])\.", # Removi o .*?\*\*
        r"A conduta correta, neste caso, é:\s*([a-dA-D])\.",
        r"Portanto, o exame mais importante para o seguimento a longo prazo da criança, considerando a suspeita clínica de glomerulonefrite pós-estreptocócica, é \*\*([a-dA-D])\.", # Removi o .*?\*\*
        r".*?, a opção mais adequada dada a situação clínica do paciente é \*\*([a-dA-D])\*\*",
        r".*?, a alternativa \*\*([a-dA-D])\*\* é a mais adequada",
        r"A alternativa \*\*([a-dA-D])\*\* é a mais apropriada",
        r".*?, a opção ([a-dA-D]) \(.*?\) é a mais compatível",
        r".*?, a conduta médica inicial correta seria:\s*([a-dA-D])",
        r"Assim, a opção mais adequada, considerando .*?, seria:\s*([a-dA-D])\.",
        r"Dada a situação descrita .*? o exame mais apropriado .*? é:\s*([a-dA-D])\.",
        r"Portanto, a opção mais adequada, levando em consideração .*? é:\s*([a-dA-D])\.",
        r"Dadas as opções e o contexto, \*\*opção ([a-dA-D]) \(.*?\)\*\* é a mais adequada",
        r"A opção \*\*([a-dA-D])\*\* é a mais coerente",
        r"A opção mais apropriada, considerando a descrição do caso, é a \*\*([a-dA-D])",
        r"A classificação correta para .*? é a opção ([a-dA-D])",
        r"A opção correta para .*? é:\s*([a-dA-D])\.",
        r"Portanto, a alternativa mais correta é:\s*\*\*([a-dA-D])",
        r"Com base .*? a alternativa \*\*([a-dA-D])\*\*.*?",
        r".*?, com base .*? o item ([a-dA-D]), \*\*(.*?)\*\*, é o mais coerente", # Mantive, mas observe o .*?
        r".*?, a opção ([a-dA-D]) é a mais correta",
        r"opção correta, .*?, é a \*\*opção ([a-dA-D])",
        r'Considerando essas evidências, o item \"([a-dA-D])\..*?\" é o diagnóstico correto',
        r"Portanto, a conduta médica adequada neste caso é a alternativa ([a-dA-D])",
        r"A resposta correta é a alternativa \*\*([a-dA-D])\*\*",
        r"Para a questão apresentada, o item que está correto é:\s*([a-dA-D])\.",
        r"Portanto, a opção ([a-dA-D]) está correta",
        r"Analisando as opções, a resposta mais adequada é a \*\*([a-dA-D])\*\*",
        r"Diante deste quadro, a conduta .*? correta seria:\s*([a-dA-D])",
        r"opção ([a-dA-D]) é a mais apropriada",
        r".*?, a hipótese diagnóstica mais provável .*? é:\s*([a-dA-D])",
        r".*?, a melhor conduta médica é a indicada no item ([a-dA-D])",
        r".*?, a opção mais adequada é a opção ([a-dA-D]):",
        r".*?, a alternativa correta é \*\*([a-dA-D])",
        r".*?, a opção mais indicada e respaldada, com base nos dados clínicos, é a \*\*([a-dA-D])",
        r"Com base na descrição clínica, a opção mais coerente é a \*\*([a-dA-D])", # Corrigi "b" para ([a-dA-D])
        r"Para essa situação, .*?, é o item \*\*([a-dA-D])\.", # Removi o .*?\*\*
        r"Com base nos .*?, a melhor conduta é a opção ([a-dA-D])",
        r".*? com base nos .*?, .*? que mais contribui para o diagnóstico é a opção ([a-dA-D])",
        r"Entre as opções, .*? \(opção ([a-dA-D])\) é geralmente preferida",
        r"Resposta: \*\*([a-dA-D])",
        r".*?, a alternativa correta seria:\s*\*\*([a-dA-D])\.",
        r".*? a opção mais apropriada para a situação descrita na questão é a opção ([a-dA-D])",
        r".*?, a alternativa ([a-dA-D]) é a mais correta",
        r".*?, a conduta médica mais indicada .*? com base .*?, é a \*\*letra ([a-dA-D])",
        r".*?, a opção mais alinhada com .*? seria:\s*\*\*([a-dA-D])",
        r"Portanto, a alternativa correta seria:\s*([a-dA-D])\.",
        r"(?:seria:|seria a:|é:|é a:|Diante disso, a opção.*?seria:)\s*\n*([a-d])\.",
        r"Dentre as opções apresentadas, .*? é a\s*\"([a-d])",
        r"é a alternativa\s*([a-dA-D])",
        r"principal hipótese diagnóstica para o quadro clínico apresentado é a letra ([a-dA-D])",
        r"descrição do caso clínico apresentado se encaixa .*?, o que corresponde ao item\s*([a-dA-D])\.$",
        r"a melhor conduta a ser realizada é (?:a|o) (?:opção|item|alternativa|letra) \s*([a-dA-D])",
        r"opção\s*([a-dA-D])\s*é a conduta imediata e mais adequada",
        r"(?:opção|item|alternativa|letra) \s*\**([a-dA-D])\** é (?:a|o) mais (?:indicada|adequada|correta|correto)",
        r"(?:opção|item|alternativa|letra) \(*\s*\**([a-dA-D])\)*\s*\** está (?:correto|correta)",
        r"hipótese diagnóstica mais provável para .*? é:\s*([a-dA-D])",
        r"com base nas informações fornecidas, a condição .*? é melhor classificada como:*\s*\**([a-dA-D])",
        r"o que corresponde (?:ao|a|à) (?:opção|item|alternativa|letra) \s*\**([a-dA-D])",
        r"(?:opção|item|alternativa|letra) correta (?:seria|é) (?:a|o) (?:opção|item|alternativa|letra)* \"*\s*\**([a-dA-D])",
        r"(?:opção|item|alternativa|letra)\s+(?:mais compatível|mais provável|mais consistente|correta|correto|mais apropriada|mais apropriado|mais adequada|mais adequado|mais indicativa|mais indicativo)\s+(?:seria|é)\s*(?:a|o)*\s*(?:opção|item|alternativa|letra)*\s*\"?\s*\**([a-dA-D])",
        r".*?(?:opção|item|alternativa|letra)\s*\"?\**([a-dA-D])\.?.*?\**\"?.*?(?:seria|é)*\s*o*\s*(?:mais lógica|mais provável|mais consistente|mais apropriada|mais apropriado|mais adequada|mais adequado|correta|correto|mais indicativa|mais indicativo)",
        r"A hipótese diagnóstica mais provável para o caso clínico descrito é (?:a|o)* (?:opção|item|alternativa|letra)*\s*\"?\**([a-dA-D])",
        r"hipótese diagnóstica mais compatível com o cenário descrito é.*?\(*(?:opção|item|alternativa|letra)\s*\"*\**([a-zA-Z])",
        r"(?:conduta|opção|item|alternativa|letra)\s+(?:mais lógica|mais provável|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)\s+(?:neste caso|nesse caso)*\s+(?:pode ser|seria|é)\s*(?:a|o)*\s*(?:opção|item|alternativa|letra)*\s*\"?\s*\**([a-dA-D])",
        r"(?:conduta|opção|item|alternativa|letra)\s+\"?\s*\**([a-dA-D])\"?\s*\**\s+(?:seria|é)\s+(?:a|o)*\s*(?:mais compatível|mais lógica|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)",
        r"(?:conduta|opção|item|alternativa|letra)\s+(?:mais compatível|mais lógica|mais provável|que melhor se encaixa|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo).*?\s+(?:pode ser|seria|é)\s+(?:a|o)*\s+(?:conduta|opção|item|alternativa|letra)*\s+\"?\**([a-dA-D])",
        r"(?:conduta|opção|item|alternativa|letra)\s+\"*\**([a-dA-D])\"*\**\s+(?:seria|é)\s+(?:a|o)*\s+(?:mais compatível|mais lógica|mais provável|recomendado|recomendada|que melhor se encaixa|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)",
        r"(?:conduta|opção|item|alternativa|letra)\s+(?:mais compatível|mais lógica|mais provável|recomendado|recomendada|que melhor se encaixa|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)\s+para.*?\s*(?:pode ser|seria|é)\s*:*\s*\**([a-dA-D])",
        r"quadro clínico descrito na questão sugere mais fortemente um diagnóstico de\s+\(*\"*\**([a-dA-D])",
        r"\s+(?:seria|é).*?\((?:opção|item|alternativa|letra)\s*([a-dA-D])\)",
        r"A intervenção terapêutica instituída nesse sentido foi\s+([a-dA-D])\)",
        r"A intervenção terapêutica instituída nesse sentido foi:\s*([a-dA-D])",
        r"(?:conduta|opção|item|alternativa|letra)*\s+(?:mais compatível|mais lógica|mais provável|recomendado|recomendada|que melhor se encaixa|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)\s+(?:pode ser|seria|é)\s+(?:a|o)*\s+(?:conduta|opção|item|alternativa|letra)*\s*:*\"*\**([a-dA-D])",
        r"\b(?:conduta|opção|item|alternativa|letra)?\s*(?:mais compatível|mais lógica|mais provável|recomendado|recomendada|que melhor se encaixa|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)?\s*(?:pode ser|seria|é)\s*(?:a|o)?\s*(?:conduta|opção|item|alternativa|letra)?\s*\(*([a-dA-D])\)*\b",
        r"(?:conforme|de acordo com|segundo)?\s+proposto\s*(?:no|na|em)\s*(?:conduta|opção|item|alternativa|letra)?\s*\**([a-dA-D])\**",
        r"(?:apenas|somente)\s+(?:a|o)\s+(?:conduta|opção|item|alternativa|letra)\s+\(?([a-dA-D])\)? se aproximou",
        r"(?:conduta|opção|item|alternativa|letra)\s+\(?\*?([a-dA-D])\*?\)?.*?(?:parece ser a|pode ser|seria|é)\s+(?:a|o)*\s+(?:mais coerente|que melhor explica|mais compatível|mais lógica|mais provável|recomendado|recomendada|que melhor se encaixa|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)",
        r"(?:conduta|opção|item|alternativa|letra)?\s+(?:mais compatível|mais lógica|mais provável|recomendado|recomendada|que melhor se encaixa|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)?\s+(?:pode ser|seria|é)\:?\s*\**([a-dA-D])\**",
        r"\**Conclusão:\s+([a-dA-D]).+",
        r"que\s+corresponde\s+(?:a|ao|à)\s+(?:conduta|opção|item|alternativa|letra)\:?\s*\**([a-dA-D])",
        r"(?:conduta|opção|item|alternativa|letra)\s+(?:mais compatível|mais lógica|mais provável|recomendado|recomendada|que melhor se encaixa|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)\s+(?:pode ser|seria|é)\s+(?:a|o)\:?\s*\**([a-dA-D])",
        r"\**situação clínica determinante para a conduta de encaminhamento à maternidade\** é \**([a-dA-D])",
        r"diagnóstico\s+(?:mais compatível|mais lógica|mais provável|recomendado|recomendada|que melhor se encaixa|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)\s+para(?:\s*a|o)?\s+paciente\s+(?:pode ser|seria|é)\s*\**([a-dA-D])(?:\.)?\**",
        r"o fator que demonstra o pior prognóstico para o quadro dessa paciente é:\s*([a-dA-D])",
        r"(?:conduta|opção|item|alternativa|letra)\s+(?:mais correta|mais correto|mais compatível|mais lógica|mais provável|que melhor se encaixa|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)\s+(?:parece ser|pode ser|seria|é):?\s*\**([a-dA-D])",
        r"(?:conduta|opção|item|alternativa|letra)\s*\**([a-dA-D])\**\s+(?:parece ser a|pode ser|seria|é)\s+(?:a|o)?\s*(?:mais coerente|que melhor explica|mais compatível|mais lógica|mais provável|recomendado|recomendada|que melhor se encaixa|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)",
        r"\b(?:opção|conduta|item|alternativa|letra)\s*\**([a-dA-D])\**\s*(?:\(|\-)?",
        r"\b(?:opção|conduta|item|alternativa|letra|hipótese diagnóstica)\s+(?:parece ser a|pode ser|seria|é)\s*\**([a-dA-D])",
        r"hipótese diagnóstica mais provável para a paciente é de \**([a-dA-Z])\)\s",
        r"(?:conduta médica inicial|conduta|opção|item|alternativa|letra)\s+(?:mais correta|mais correto|mais compatível|mais lógica|mais provável|que melhor se encaixa|mais consistente|mais apropriada|mais apropriado|correta|correto|mais adequada|mais adequado|mais indicativa|mais indicativo)\s+(?:parece ser|pode ser|seria|é):?\s*(?:encaminhar o paciente para uma|encaminhar a paciente para uma)?:?\s*([a-zA-D])\)",
        r"^([a-dA-D])\.*?\s",
      ]

    for pattern in patterns:
      match = re.search(pattern, resposta, re.IGNORECASE)

      if match:
          # Extract the content of the first capturing group
          return match.group(1).upper()

    return "VOID"


In [22]:
# Inicializando variáveis
respostas = {"questao": np.linspace(1, 100, 100, dtype=int)}

for sheet_name in sheet_names:
    file_name = f"respostas-{sheet_name}.xlsx"

    # Ler dados do Excel
    df_respostas = pd.read_excel(file_name)
    lista_perguntas = df_respostas[f'Perguntas-{sheet_name}'].to_list()
    lista_respostas = df_respostas[f'Repostas-{sheet_name}'].to_list()
    lista_gabaritos = df_respostas[f'Gabarito-{sheet_name}'].to_list()

    # Inicializando listas para armazenar respostas e gabaritos
    name_resposta = f"respostas_{sheet_name}"
    name_gabarito = f"gabarito_{sheet_name}"
    respostas[name_resposta] = []
    respostas[name_gabarito] = []

    print(f"Exame {sheet_name}")

    for resposta, gabarito in zip(lista_respostas, lista_gabaritos):
        respostas[name_gabarito].append(gabarito.strip())

        if gabarito not in ["IMAGEM", "TABELA", "NULA"]:
            letra = extrair_opcao(resposta)
            respostas[name_resposta].append(letra)
            print(f"Questão {len(respostas[name_resposta])}: {letra.strip()}")
        else:
            respostas[name_resposta].append(gabarito)

print("Processamento concluído!")

Exame 2024.01
Questão 1: D
Questão 2: D
Questão 3: D
Questão 4: D
Questão 5: D
Questão 6: D
Questão 8: D
Questão 9: D
Questão 10: D
Questão 11: D
Questão 12: D
Questão 15: D
Questão 16: D
Questão 18: D
Questão 19: D
Questão 22: D
Questão 23: D
Questão 24: D
Questão 25: D
Questão 26: D
Questão 27: D
Questão 28: D
Questão 29: D
Questão 30: D
Questão 31: D
Questão 32: D
Questão 33: D
Questão 34: D
Questão 35: D
Questão 37: D
Questão 39: D
Questão 40: D
Questão 41: D
Questão 42: D
Questão 44: D
Questão 45: D
Questão 47: D
Questão 48: D
Questão 49: D
Questão 51: D
Questão 52: D
Questão 53: D
Questão 54: D
Questão 55: D
Questão 56: D
Questão 57: D
Questão 58: D
Questão 59: D
Questão 60: D
Questão 61: D
Questão 62: D
Questão 63: D
Questão 64: D
Questão 65: D
Questão 66: D
Questão 67: D
Questão 68: D
Questão 70: D
Questão 71: D
Questão 72: D
Questão 73: D
Questão 74: D
Questão 75: D
Questão 76: D
Questão 78: D
Questão 79: D
Questão 80: D
Questão 81: D
Questão 82: D
Questão 84: D
Questão 85: D


In [23]:
df = pd.DataFrame(respostas)
df

,questao,respostas_2024.01,gabarito_2024.01
0,1,D,A
1,2,D,A
2,3,D,C
3,4,D,D
4,5,D,A
...,...,...,...
95,96,D,A
96,97,D,C
97,98,D,B
98,99,D,C


In [24]:
df.to_excel('resposta_gabarito.xlsx', index=False)

In [25]:
from google.colab import files

files.download('resposta_gabarito.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>